In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn.linear_model  as sk  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from helper import *
import tensorflow as tf

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator

from keras.layers import Dense
from keras.models import Model, Sequential
from keras import initializers, optimizers
from keras import backend as K

Using TensorFlow backend.


# Stuff that needs to be done

- Find another framework.
    - Sklearn
    - Spark
    - Tensorflow/R
- Algorithms
    - Linear Regression
    - Logistic Regression
    - Maybe SVM /Random forest
- Find two metrics 
    - RMSE 
    - Some other
- Find another dataset 
    - Sum data with noise 
    -  Maybe housing data set
    

## Import DataSet

In [2]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [3]:
get_data(all_urls) # retrieves the data if there is no data folder

In [4]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/housing dataset.csv" # has more than 30 features
# need one more
# what a brilliant idea to name files with space

In [5]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

## Load datasets sum_noise

In [6]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise.head(n=1)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


### Create same base line for Spark, Keras, Sklearn implementation
- Using the same feature scaled data for all implementation
- Same number iterations
- Same cost function for minimising(for gradient descent)

# Need to check the ordering of the categorical to numerical

In [111]:
# Remove 'Instance' as it simply represents the row number



sumdata_noise.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise.loc[:, ['Noisy Target']]
# Extract 'Nosiy Target Class' as regression target
sumdata_noise_classif_Y = sumdata_noise.loc[:, ['Noisy Target Class']] 
 
from sklearn import preprocessing
le = preprocessing.LabelEncoder() 
 
sumdata_noise_classif_Y ['Noisy Target Class'] = le.fit_transform( sumdata_noise_classif_Y ['Noisy Target Class'].astype('str')) 
 
# Extract rest columns as explananatory variables
sumdata_noise_X =sumdata_noise.drop(['Instance','Noisy Target Class','Noisy Target'
                                    ], axis = 1) 
 
sumdata_noise_classif_Y.shape
sumdata_noise_X.shape

(968135, 10)

### Feature scaling
- Keeping them as dataframes (transform returns ndarray)

In [112]:
scX = StandardScaler()
scY = StandardScaler()
columns = sumdata_noise_X.columns

sumdata_noise_X[columns] = scX.fit_transform(sumdata_noise_X)
# columns = sumdata_noise_reg_Y.columns
# sumdata_noise_reg_Y[columns] = scX.fit_transform(sumdata_noise_reg_Y)


sumdata_noise_scaled = pd.concat([sumdata_noise_X,sumdata_noise_reg_Y])

In [113]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-9-8030b2fc72d9>:1 

In [145]:
def spark_logistic_regression_new( X_train, X_test, y_train, y_test):
    """
        this uses the dataframe api
    """
    from pyspark.ml import Pipeline
    from pyspark.ml.classification import LogisticRegression
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    # spark needs a target column
    index = X_train.index.values
    
    
    ## SHOULD FIND A BETTER WAY
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    train_data = pd.concat([X_train,y_train], axis =1)
    test_data  = pd.concat([X_test,y_test], axis =1)
   
    train_data.rename(columns={'Noisy Target Class': 'label'}, inplace=True)
    test_data.rename(columns={'Noisy Target Class': 'label'}, inplace=True)
    
 
    train_data = sqlContext.createDataFrame(train_data)
    test_data = sqlContext.createDataFrame(test_data) 
   
   
    assembler = (VectorAssembler()
        .setInputCols(train_data.columns[:-1]) # everything excluding target
        .setOutputCol("features"))

#     output = assembler.transform(train_data)
#     output.select("features", "label").show(truncate=False)
    lr = LogisticRegression(maxIter=10, regParam=0.01)

    pipeline = Pipeline(stages=[assembler, lr])
    model = pipeline.fit(train_data)
    
    
    predictions = model.transform(test_data) 
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
    accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
    precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
    print(accuracy)
#     #print( evaluator.evaluate(model.transform(test_data), {evaluator.metricName: "areaUnderROC"}))
#     from sklearn.metrics import accuracy_score
#     accuracy  = accuracy_score(y_test, model.predict(X_test))
#     precision  = precision_score(y_test, model.predict(X_test))
#     return RMSE

In [146]:
def model_test_cross( data_x, data_y, dataset_name, algorithm): 
    algorithm_n = algorithm.__name__
    errors = np.zeros((1, 10)) #  
    for chunk in data_chunks:
        # if chunk is greater than the no. of examples
        if chunk > sumdata_noise.shape[0]: 
            break

        X = data_x.head(n = chunk)
        y = data_y.head(n = chunk) 
        
        kf = KFold(n_splits=10)
        
        
        for (i, (train_index, test_index)) in enumerate(kf.split(X)):
            
            
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            # accumulating errors for each fold
            errors[0][i] = algorithm( X_train, X_test, y_train, y_test) 
        
        # calculates mean across rows
      
        mean_error = np.mean(errors, axis=0)
         
        
        print ("Algorithm: {}  Data set: {} Chunk Size: {} Error: {}".format(algorithm_n,
                                                                             dataset_name,
                                                                             chunk, mean_error[0]))
        

In [147]:
model_test_cross( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", spark_logistic_regression_new)

0.9
1.0
1.0
1.0
1.0
1.0
1.0
0.9
0.9
0.9
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 100 Error: nan
1.0
0.92
0.96
0.96
0.86
0.98
0.94
0.98
0.98
0.92
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 500 Error: nan
0.96
0.96
0.92
0.96
0.95
0.92
0.92
0.99
0.94
0.94
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 1000 Error: nan
0.954
0.942
0.934
0.968
0.942
0.952
0.934
0.946
0.942
0.944
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 5000 Error: nan
0.95
0.951
0.95
0.938
0.946
0.943
0.944
0.953
0.962
0.953
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 10000 Error: nan
0.9472
0.9516


KeyboardInterrupt: 

### 70/30 split 

In [117]:
def model_test_split( data_x, data_y, dataset_name, algorithm): 
    algorithm_n = algorithm.__name__
  
    for chunk in data_chunks:
        # if chunk is greater than the no. of examples
        if chunk > sumdata_noise.shape[0]: 
            break

        X = data_x.head(n = chunk)
        y = data_y.head(n = chunk) 
        
        kf = KFold(n_splits=10)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
        
       
            # accumulating errors for each fold
        error = algorithm( X_train, X_test, y_train, y_test) 
        
        print ("Algorithm: {}  Data set: {} Chunk Size: {} Error: {}".format(algorithm_n,
                                                                             dataset_name,
                                                                             chunk, error))
        

In [118]:
model_test_split( sumdata_noise_X, sumdata_noise_classif_Y, "Sum data with noise", spark_logistic_regression_new)

Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 100 Error: None
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 500 Error: None
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 1000 Error: None
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 5000 Error: None
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 10000 Error: None
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 50000 Error: None
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 100000 Error: None
Algorithm: spark_logistic_regression_new  Data set: Sum data with noise Chunk Size: 500000 Error: None
